# 15.Geospatial Queries in MongoDB

### Main Concepts:

1. **Geospatial Queries**:
   - Allow querying data based on location.
   - Supported by MongoDB.

2. **GeoJSON Format**:
   - A format used to store geospatial data in MongoDB.
   - Types of GeoJSON objects: Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon.
   - Each type has specific structure and requirements.

### Types of GeoJSON Objects:

1. **Point**:
   - Single point represented by latitude and longitude.
   - Example: `{ type: "Point", coordinates: [ longitude, latitude ] }`

2. **LineString**:
   - Line represented by two or more coordinates joining two points.
   - Example: `{ type: "LineString", coordinates: [ [ longitude1, latitude1 ], [ longitude2, latitude2 ] ] }`

3. **Polygon**:
   - Shape represented by multiple points.
   - Example: `{ type: "Polygon", coordinates: [ [ longitude1, latitude1 ], [ longitude2, latitude2 ],... ] }`

### Saving GeoJSON Data in MongoDB:

1. **Format**:
   - Use GeoJSON format.
   - Specify type of GeoJSON object and coordinates.

### Indexing Geospatial Data:

1. **Types of Indexing**:
   - **2d Sphere Indexing**: For spherical coordinates (e.g., globe).
   - **2d Flat Indexing**: For flat coordinates (e.g., map).

### Example of Saving GeoJSON Data:

- Using `db.places.insertOne()` method.
- Data saved with a name and location (GeoJSON point with coordinates).

### Key Points:

1. **Geospatial Queries**:
   - Used to find locations near a specific point.
   - Represent locations as points on a 2D sphere or flat surface.

2. **Near Operator**:
   - Finds locations near a specific point.
   - Requires a geometry object with GeoJSON data and coordinates (longitude and latitude).
   - Can specify distance range with `maxDistance` and `minDistance` options.
   - Distances calculated in meters.
   - Returns locations within the specified distance range.

3. **Indexing for Efficient Querying**:
   - Create an index for the geospatial field using the `createIndex` method with the `2dsphere` option.

### Applications:

- Geospatial queries can be used to find nearby points of interest, such as restaurants near a user's location.

### `$near`

- **Definition**: The `$near` operator selects documents based on their proximity to a specified point.
- **Usage**: It finds documents near a given point in a 2D spherical coordinate system (like the Earth's surface).
- **GeoJSON Format**: Specify the point using GeoJSON format with `type: "Point"` and `coordinates: [longitude, latitude]`.
- **Example**: Find all points within a certain distance from a central point.

### `$geoWithin`

- **Definition**: The `$geoWithin` operator selects documents with geospatial data that falls entirely within a specified geometry.
- **Usage**: It checks if the geometry field (like a point, line, or polygon) is within a specified shape (like a polygon or circle).
- **GeoJSON Format**: Define the shape using GeoJSON format with `type` (e.g., `"Polygon"`, `"Circle"`), and `coordinates`.
- **Example**: Find all points or shapes contained within a specified polygon or boundary.



### Wroking with points

In [12]:
from pymongo import MongoClient  
    
# Creation of object MongoClient  
client = MongoClient()  
    
# Connect with the port number and host  
client = MongoClient("mongodb://localhost:27017/")  
    
# Access database  
mydatabase = client['Geospace']  
    
# Access collection of the database  
collection = mydatabase['Locations']  

# Define the data
data = {
    "name": "Ahamed Cabs",
    "location": {
        "type": "Point",
        "coordinates": [
            8.9989476,   # Latitude first
            77.2213398  # Longitude second
            
        ]
    }
}

# Insert the data into the collection
collection.insert_one(data)

InsertOneResult(ObjectId('668e34db5ebe7e7915cce456'), acknowledged=True)

In [3]:
collection.find_one()

{'_id': ObjectId('668e31065ebe7e7915cce452'),
 'name': 'Ahamed Cabs',
 'location': {'type': 'Point', 'coordinates': [77.2213398, 8.9989476]}}

In [6]:
nearby_locations = collection.find({
    "location": {
        "$near": {
            "$geometry": {
                "type": "Point",
                "coordinates": [
                    8.9989476,    # Latitude first
                    77.2213398  # Longitude second
                ]
            }
        }
    }
})

In [7]:
for location in nearby_locations:
    print(location)  ## Error because we need to create the index first

OperationFailure: error processing query: ns=Geospace.LocationsTree: GEONEAR  field=location maxdist=1.79769e+308 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=Geospace.LocationsTree: GEONEAR  field=location maxdist=1.79769e+308 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [8]:
collection.create_index([("location", "2dsphere")])

'location_2dsphere'

In [21]:
nearby_locations = collection.find({
    "location": {
        "$near": {
            "$geometry": {
                "type": "Point",
                "coordinates": [
                    # 8.9990307,77.2211693  MKB School
                    8.9990307,   # Latitude first
                    77.2211693  # Longitude second
                
                ]
            },
            
            "$maxDistance": 20 # Takes 20 meters surrondings
        }
    }
})

In [22]:
for location in nearby_locations:
    print(location)

{'_id': ObjectId('668e34db5ebe7e7915cce456'), 'name': 'Ahamed Cabs', 'location': {'type': 'Point', 'coordinates': [8.9989476, 77.2213398]}}


In [11]:
## Add more records

data = {
    "name": "Canara bank",
    "location": {
        "type": "Point",
        "coordinates": [
            8.9991189,  # Latitude first
            77.2192646    # Longitude second
        ]
    }
}

# Insert the data into the collection
collection.insert_one(data)

InsertOneResult(ObjectId('668e345a5ebe7e7915cce453'), acknowledged=True)

### Wroking with polygons

In [39]:
from pymongo import MongoClient

# Creation of object MongoClient
client = MongoClient()

# Connect with the port number and host
client = MongoClient("mongodb://localhost:27017/")

# Access database
mydatabase = client['Geospace']

# Access collection of the database
collection = mydatabase['Polygons']
# Insert example points data
point_data = [
    {
        "name": "Point A",
        "location": {
            "type": "Point",
            "coordinates": [1, 1]
        }
    },
    {
        "name": "Point B",
        "location": {
            "type": "Point",
            "coordinates": [2, 2]
        }
    },
    {
        "name": "Point C",
        "location": {
            "type": "Point",
            "coordinates": [3, 3]
        }
    }
]

# Insert the point data into the collection
collection.insert_many(point_data)

InsertManyResult([ObjectId('668e3b575ebe7e7915cce467'), ObjectId('668e3b575ebe7e7915cce468'), ObjectId('668e3b575ebe7e7915cce469')], acknowledged=True)

In [40]:
# Create a 2dsphere index on the 'location' field
collection.create_index([("location", "2dsphere")])

'location_2dsphere'

In [45]:
# Define the polygon coordinates
polygon_coordinates = [
    [0, 0],
    [0, 5],
    [5, 5],
    [5, 0],
    [0, 0]
]

# Define the query to find points both within and near the polygon
query = {
    "location": {
        "$geoWithin": {
            "$geometry": {
                "type": "Polygon",
                "coordinates": [polygon_coordinates]
            }
        }
    },
    "location": {
        "$near": {
            "$geometry": {
                "type": "Point",
                "coordinates": [2, 2]
            },
            "$maxDistance": 1000  # Adjust the maximum distance as needed
        }
    }
}

# Perform the query
results = collection.find(query)

# Print the results
for result in results:
    print(result)

{'_id': ObjectId('668e3b575ebe7e7915cce468'), 'name': 'Point B', 'location': {'type': 'Point', 'coordinates': [2, 2]}}


## Coordinate System

1. **Latitude and Longitude**:
   - Coordinates are specified as `[longitude, latitude]` in GeoJSON format.
   - Longitude ranges from -180 to 180 degrees.
   - Latitude ranges from -90 to 90 degrees.

### Geospatial Indexing

1. **2dsphere Index**:
   - Used for spherical coordinates (representing the Earth's surface).
   - Supports querying data in GeoJSON format.
   - Uses spherical geometry calculations to determine distances and relationships.

### Distance Calculations

1. **Great-Circle Distance**:
   - For spherical coordinates, MongoDB uses the Haversine formula to calculate the great-circle distance between two points.
   - The formula accounts for the Earth's curvature and provides the shortest distance over the Earth's surface.

2. **Flat Plane Distance**:
   - For flat coordinates, a simple Euclidean distance formula is used.

### Haversine Formula (Spherical)

The Haversine formula calculates the distance between two points on a sphere given their longitudes and latitudes:

\[ a = \sin^2\left(\frac{\Delta \phi}{2}\right) + \cos(\phi_1) \cdot \cos(\phi_2) \cdot \sin^2\left(\frac{\Delta \lambda}{2}\right) \]

\[ c = 2 \cdot \text{atan2}\left(\sqrt{a}, \sqrt{1-a}\right) \]

\[ d = R \cdot c \]

Where:
- \( \phi \) is latitude in radians.
- \( \lambda \) is longitude in radians.
- \( \Delta \phi \) and \( \Delta \lambda \) are the differences in latitude and longitude.
- \( R \) is the Earth's radius (mean radius = 6,371 km).
- \( d \) is the distance between the two points.



#### Prepared By,
Ahamed Basith